# Factors tutorial

This tutorial covers the atomic units of The Sandbox Environment for Generalizable Agent Research (SEGAR): the factors.
Factors make up the underlying state space, and their values make up the underlying states.
Factors are _types_, and many of them inherit from built-in types, such as `float`, `int`, `bool`, etc.

### Factors
Here are some basic examples of creating and manipulating factors:

In [1]:
from segar.factors import Charge, Mass, Alive, Position, Factor
c1 = Charge(0.3)
c2 = Charge(0.3)
m = Mass(0.3)
a = Alive(True)
p = Position([0.1, 0.1])

In [2]:
print(c1, c2)

Charge(0.3) Charge(0.3)


Factors can take on values, so they can be compared and operated on, but they are _unique_

In [3]:
print(c1 == c2, c1 is c2, c1 == m, c1 + m, p + m)

True False True 0.6 Position([0.4 0.4])


The base type `Factor` is a generic type:

In [4]:
f = Factor[float](0.4)
print(f, f + c1)

Factor(0.4) Charge(0.7)


### Factor Containers
In order to be useable by the sim, sets factors must be encapsulated in a container called a `FactorContainer`:

In [5]:
from factors import FactorContainer
fc = FactorContainer({Charge: 0.3, Position: [.1, .3], Alive: False})
print(fc)

FactorContainer({Charge: Charge(0.3), Position: Position([0.1 0.3]), Alive: Alive(False)})


Factor containers are basically dictionaries with Factor type keys and Factor values. We can check if a FactorContainer contains a factor or not:

In [6]:
print(fc.has_factor(Charge), fc.has_factor(Mass))

True False


Changing factors of their containers is protected:

In [7]:
try:
    fc[Charge] = 0.4
    raise RuntimeError('Should not get here')
except ValueError as e:
    print('Error message:', e)


Error message: In-place operations protected.


But in-place operations can be allowed:

In [8]:
with fc.in_place():
    fc[Charge] += 0.4
print(fc[Charge])

Charge(0.7)


### Random factor generation
SEGAR comes with factor generation from random numbers. This can help generate distributions of factors used for initialization, or even can be used to generate stochastic transition functions:

In [9]:
from segar.factors import UniformNoise, GaussianNoise, GaussianNoise2D
u1 = UniformNoise(0., 1.)  # Uniform from 0 to 1
u2 = UniformNoise(0.1, 1.)  # Uniform from 0.1 to 1
g1 = GaussianNoise(0., 0.1)  # Normal distribution with 0 mean and 0.1 std
g2 = GaussianNoise(0.1, 0.2)  # Normal distribution with 0.1 mean and 0.2 std
g2D = GaussianNoise2D([0., 0.1], [0.1, 0.2])  # 2D normal distribution

print(u1.sample(), g1.sample(), g2D.sample())

Factor(0.7174490241465336) Factor(-0.0519678877226635) Factor([ 0.30422966 -0.48938501])


SEGAR allows the researcher to compare distributions (via the Wasserstein distance), ultimately to measure how different different sets of tasks are (say between train and test in a generalization experiment).

In [11]:
print(u1.w2_distance(u1), u1.w2_distance(u2), u1.w2_distance(g1), g1.w2_distance(g1), g1.w2_distance(g2))

0.000995363069815011 0.0586407279494164 0.53618311310831 0.0 0.02715728752538099


Note that the distance of `u1` with itself is not measured as zero. This is because we are using a sample-based approximation for the Wasserstein-2 in most cases. More accurate measurement of the W2 can be accomplished by increasing the number of samples used. This increases computational cost; however, as these measures are only intended to be used to compare tasks, this can be done separate of running agents for analysis and results.

There are a number of distributional methods available for each of these noise factors, such as cdfs, pdfs, etc. One can even access the corresponding scipy distribution objects for other distributional analysis:

In [13]:
print(u1.scipy_dist)

In [14]:
print(u1.cdf(u1.sample().value))

0.38958346068677663
